The purpoes of this notebook is to create a metadata file the includes all the subjects in differnet version of PPMI dataset on the BIC server. It includes:

    1. PPMI_ver-sdMRI # subjects with both T1 and DWI

    2. PPMI_ver-T1    # subjects with T1.

In [1]:
from pathlib import Path
import pandas as pd

#PATH
codes_dir = Path("/scratch/mr_proc")
# in
sdMRI_bids_file = codes_dir / "metadata" / "ppmi" / "participants_sdMRI.tsv"  # participants from bids dataset
t1_bids_file    = codes_dir / "metadata" / "ppmi" / "participants_T1.tsv"     # participants from bids dataset
meta_data_file  = codes_dir / "metadata" / "ppmi" / "PPMI_all_T1_7_28_2022.csv"  # PPMI all subejct meta data
# out
participant_file    = codes_dir / "metadata" / "ppmi" / "participants.csv"  

In [2]:
sdMRI_df = pd.read_csv(sdMRI_bids_file, sep='\t')
t1_df    = pd.read_csv(t1_bids_file, sep='\t')
print('N subjects in sdMRI BIDS: ', len(sdMRI_df.participant_id.unique()))
print('N subjects in T1    BIDS: ', len(t1_df.participant_id.unique()))

N subjects in sdMRI BIDS:  560
N subjects in T1    BIDS:  1239


In [3]:
meta_df  = pd.read_csv(meta_data_file)
meta_df=meta_df.loc[:,['Subject','Age', 'Sex', 'Group','Visit']].drop_duplicates()
meta_df=meta_df.sort_values(by=['Subject','Visit'], ascending=True).drop_duplicates(subset='Subject', keep='first')
meta_df.loc[:,'Subject']=meta_df.loc[:,'Subject'].apply(lambda x: 'sub-'+str(x))
print('Number of subjects in all data: ', len(meta_df.Subject.unique()))
meta_df.head(5)

Number of subjects in all data:  1277


,Subject,Age,Sex,Group,Visit
3927,sub-01218AUG16,56,X,Phantom,0
3926,sub-01221JUL16,56,X,Phantom,0
3923,sub-08827APR11,39,M,Phantom,1
3908,sub-100001,67,M,PD,1
3902,sub-100005,53,M,PD,1


In [4]:
participant_list=list(pd.concat([sdMRI_df, t1_df]).participant_id.unique())
participant_df = meta_df[meta_df['Subject'].isin(participant_list)]
#participant_df.loc[:,['participant_id','age','sex','group']]=participant_df.loc[:,['Subject','Age','Sex','Group']]
participant_df.rename(columns = {'Subject':'participant_id', 'Age':'age', 'Sex':'sex', 'Group':'group'}, inplace = True)
participant_df=participant_df.drop(columns=['Visit'])
print('Total number of mered subjects: ', len(participant_df.participant_id.unique()))
print('Total number of merged groups: ', len(participant_df.group.unique()), ', and they are: ')
print(participant_df.group.unique())
participant_df.head(5)

Total number of mered subjects:  1240
Total number of merged groups:  7 , and they are: 
['Phantom' 'PD' 'Prodromal' 'Control' 'SWEDD' 'GenReg Unaff' 'AV133']


/tmp/ipykernel_32514/2139515272.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participant_df.rename(columns = {'Subject':'participant_id', 'Age':'age', 'Sex':'sex', 'Group':'group'}, inplace = True)


,participant_id,age,sex,group
3927,sub-01218AUG16,56,X,Phantom
3926,sub-01221JUL16,56,X,Phantom
3923,sub-08827APR11,39,M,Phantom
3908,sub-100001,67,M,PD
3902,sub-100005,53,M,PD


In [17]:
# Save file
participant_df.to_csv(participant_file, index=False)